In [1]:
import sys
import json
import joblib
import gc
from tqdm import tqdm
import numpy as np
import os
from typing import List, Dict, Tuple
import yaml

# TO CHANGE
BASEDIR = "../"
sys.path.insert(0, BASEDIR)

from src.kg_model import KnowledgeGraphModel

/home/dzigen/Desktop/PersonalAI/.pai_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
GRAPH_DRIVER_CONFIG_PATH = "/home/dzigen/Desktop/PersonalAI/Personal-AI/data/knowledge_graphs/diaasqa/gigachat_full/graph_config"
EMBEDDINGS_DRIVER_CONFIG_PATH = "/home/dzigen/Desktop/PersonalAI/Personal-AI/data/knowledge_graphs/diaasqa/gigachat_full/embeddings_config"

In [3]:
graph_config = joblib.load(GRAPH_DRIVER_CONFIG_PATH)
embed_config = joblib.load(EMBEDDINGS_DRIVER_CONFIG_PATH)

# !!! IMPORTANT !!!
graph_config.driver_config.db_config.need_to_clear = False
embed_config.nodesdb_driver_config.db_config.need_to_clear = False
embed_config.tripletsdb_driver_config.db_config.need_to_clear = False
# !!! IMPORTANT !!!

# fixing paths to vector dbs
embed_config.embedder_config.model_name_or_path = '/'.join(embed_config.embedder_config.model_name_or_path.split("/")[2:])
embed_config.nodesdb_driver_config.db_config.path = '/'.join(embed_config.nodesdb_driver_config.db_config.path.split("/")[2:])
embed_config.tripletsdb_driver_config.db_config.path = '/'.join(embed_config.tripletsdb_driver_config.db_config.path.split("/")[2:])

In [4]:
kg_model = KnowledgeGraphModel(
    graph_config=graph_config,
    embeddings_config=embed_config)

print(kg_model.embeddings_struct.vectordbs['nodes'].count_items())
print(kg_model.embeddings_struct.vectordbs['triplets'].count_items())
print(kg_model.graph_struct.db_conn.count_items())

No sentence-transformers model found with name ../models/intfloat/multilingual-e5-small. Creating a new one with mean pooling.


49597
44328
{'triplets': 182838, 'nodes': 49597}


In [13]:
kg_model.graph_struct.db_conn.item_exist("0af4b6a7ca38cae47b45fa235992d2a7", id_type='triplet')

True

In [18]:
kg_model.graph_struct.db_conn.item_exist("523d6f983a7a394769e93298aff4dc09", id_type='relation')

True

In [25]:
import json

In [29]:
b = json.dumps("after findx2 came out , it was the same group of people who said that it was too expensive not to buy it and said that there 's no find after the eyebrows were gone [ eat melon ]")

In [31]:
json.loads(b)

"after findx2 came out , it was the same group of people who said that it was too expensive not to buy it and said that there 's no find after the eyebrows were gone [ eat melon ]"

In [27]:
output[0].end_node.name

"after findx2 came out , it was the same group of people who said that it was too expensive not to buy it and said that there 's no find after the eyebrows were gone [ eat melon ]"

In [32]:
extracted_triplets = joblib.load("/home/dzigen/Desktop/PersonalAI/Personal-AI/data/knowledge_graphs/diaasqa/gigachat_full/extracted_triplets")

In [38]:
base_triplet.id

'216c03c478a57644f72da2c57fccb7cc'

In [36]:
base_triplet = None
for triplets in extracted_triplets:
    flag = False
    for triplet in triplets:
        if triplet.id == "216c03c478a57644f72da2c57fccb7cc":
            base_triplet = triplet
            flag = True
            break
    if flag:
        break

In [44]:
base_triplet.stringified

'12.11.2019: "after findx2 came out , it was the same group of people who said that it was too expensive not to buy it and said that there \'s no find after the eyebrows were gone [ eat melon ]"'

In [50]:
base_triplet.end_node.name

'"after findx2 came out , it was the same group of people who said that it was too expensive not to buy it and said that there \'s no find after the eyebrows were gone [ eat melon ]"'

In [52]:
json.dumps(base_triplet.end_node.name)

'"\\"after findx2 came out , it was the same group of people who said that it was too expensive not to buy it and said that there \'s no find after the eyebrows were gone [ eat melon ]\\""'

In [49]:
output[0].end_node.name

"after findx2 came out , it was the same group of people who said that it was too expensive not to buy it and said that there 's no find after the eyebrows were gone [ eat melon ]"

In [53]:
raw_output = kg_model.graph_struct.db_conn.execute_query(
    'MATCH (n1)-[rel]->(n2) WHERE rel.t_id = "216c03c478a57644f72da2c57fccb7cc" RETURN n1, rel, n2')

In [57]:
raw_output[0]['n2']['name']

'"after findx2 came out , it was the same group of people who said that it was too expensive not to buy it and said that there \'s no find after the eyebrows were gone [ eat melon ]"'

In [11]:
output = kg_model.graph_struct.db_conn.read(['699959fd97a0abefad26f51770c461f4'])

PARSED_TRIPLETS: 
* triplet_id = 699959fd97a0abefad26f51770c461f4
	 s_node: id = 554d7f8fe5d6c4900d289209bdd739ac; str_id = 554d7f8fe5d6c4900d289209bdd739ac
	 rel: id = 1e38f3ebe6f06e1395c1df3b4a9524f5; t_id = 699959fd97a0abefad26f51770c461f4 str_id = 1e38f3ebe6f06e1395c1df3b4a9524f5
	 e_node: id = 1e38f3ebe6f06e1395c1df3b4a9524f5; str_id = 1e38f3ebe6f06e1395c1df3b4a9524f5


In [12]:
output

[Triplet(start_node=Node(name='opinion', type=<NodeType.object: 'object'>, prop={'name': 'opinion', 'str_id': '554d7f8fe5d6c4900d289209bdd739ac'}, stringified=None, id='554d7f8fe5d6c4900d289209bdd739ac'), relation=Relation(name='episodic', type=<RelationType.episodic: 'episodic'>, prop={'t_id': '699959fd97a0abefad26f51770c461f4', 'name': 'episodic', 'str_id': '1e38f3ebe6f06e1395c1df3b4a9524f5'}, id='1e38f3ebe6f06e1395c1df3b4a9524f5'), end_node=Node(name="Here is the generated dialog:\nKathryn: I'm so disappointed with the camera of my K30pro, it's really bad.\nLaura: Oh, I have to disagree! I think the K30pro takes amazing photos, the quality is so high!\nKyle: I have to agree with Kathryn, the camera is really subpar. I've had so many bad experiences with it.\nWinifred: I recently got the IQOO8pro and I have to say, the algorithm is so impressive. It makes all the difference in photo quality.\nKathryn: Yeah, I wish my K30pro had a better camera. It's really frustrating.\nLaura: Maybe 

In [18]:
from src.utils.data_structs import Triplet, Node, NodeType, RelationType, Relation

In [21]:
kg_model.graph_struct.db_conn.item_exist('dfbddbb1fb2843828d5bf8b46c9a055a')

True

In [19]:
tmp_triplet = Triplet(
    start_node=Node(name='people', type='object', prop={}, stringified=None, id='1'), 
    relation=Relation(name='', type='hyper', prop={'t_id': 'dfbddbb1fb2843828d5bf8b46c9a055a', 'name': 'hyper', 'str_id': 'faef901af05a523760a11ba200cfaa76'}, id='a25def06266ee25f3f43dccb0ae78cf2'), 
    end_node=Node(name='people not being optimistic about pro version of camera', type='hyper', prop={'time': '8.6.2018', 'str_id': 'faef901af05a523760a11ba200cfaa76'}, stringified=None, id='1'), stringified=None, id='dfbddbb1fb2843828d5bf8b46c9a055a')

In [13]:
base_ids = ['d6e373548b7849fd291a30dc39168322', 'a25def06266ee25f3f43dccb0ae78cf2', '7744abffdc1ecae1a0628cb81b3a6306', '7d3779481445043622b21e8f22cbd11b', 'b991752df7d51c9d7bd37d7fc5d18cb3', '8fa973870ecab1c4ac9f9e9be2d8da75', '9429f4448088126cada0c39bc2f52618', '07f0099fb4f53daa948997ba6aff58dd', '4d568205abebbf73af29bb91ded0ad43', '273de39c346438229af877d161a1d57a', '399f9649167bfe8d573dc07604badc89', '11f27d9882be6557dd9ee0452fd63ad4', '6e913c8ecf2d9517bb73f930eb821432', '7e1ef21b011a83869085bed494b90547', '727fc606ad0564df55eaf377c5728608', '8bad9f0111caf608878866eff60fe298', 'aaf86f59e307f6e6e786101eccc64f95', 'bd37bf72e84df6fe0652773cc19312be', '536a9b96bf86760b2e3726a457cfc632', '144baeab869a2006d2d6add888578189', '2a9b420a91c187e5e55ede37c4153efe', 'acb90ec7c71a5370a358a275c96e8774', '8372d37c1fba8a20c4941552f1ac6d94', '6ced735f5852eec94c934d26a40eb95c', '96a4f67af669e576890a979df8c45c04', 'eaa92d702609164bba99d50121aa11a2', 'c6c264ef57decdc9a11a5ac658bab456', 'e98dc467005f8aa8721acc5648e759d6', '1870de767d8e4138d5098b79625c48e9', 'b45a8cf1e88c5323981daf8fa5d073f5', '476625ab99d5efb086f8c74b1c7ee209', 'c2a38a10dcc01e1424c3b340074375cc', '2b76b9057de8cc1617764de87ea1bda5', '30904b8c9e22b21e526cabc29aaef938', '5b6bd07b0e25501c4677a0c78cf8b594', '22b12959b6d8b327eae2384b9dcd70c7', '51f3a57de0de7f1de17eb65e04e0f158']
for id in base_ids:
    print(kg_model.graph_struct.db_conn.item_exist(id, id_type='relation'))

True
False
True
True
True
True
True
False
True
False
True
False
False
False
True
True
True
False
False
False
False
False
False
False
False
False
True
False
True
True
True
True
True
True
True
True
True


In [8]:
for id in base_ids:
    print(kg_model.embeddings_struct.vectordbs['triplets'].item_exist(id))

True
False
False
True
True
True
True
False
True
False
True
False
False
False
True
True
True
False
False
False
False
False
False
False
False
False
True
False
True
True
True
True
True
True
True
True
True
